<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [2]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [3]:
# Veuillez écrire votre code ici.
print("Traitements disponibles :", nlp.pipe_names)
for pipe in nlp.pipe_names:
    if pipe not in {"tok2vec", "morphologizer","parser"}:
        nlp.disable_pipe(pipe)

print("Traitements activés :", nlp.pipe_names)

Traitements disponibles : ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Traitements activés : ['tok2vec', 'morphologizer', 'parser']


In [3]:
from spacy.lang.fr.examples import sentences
s1 = sentences[2] # prenons la 3e phrase comme exemple

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [4]:
# Veuillez écrire votre code et votre réponse ici.
doc = nlp(s1)
print(doc.text)
for token in doc:
    print(f"{token.text} [{token.pos_}, {token.dep_}, head: {token.head.text}]")


# Il manquait le head dans le code précédent. Pourquoi c'est important ? Car il permet de savoir quel est le mot qui gouverne le mot courant.

San Francisco envisage d'interdire les robots coursiers sur les trottoirs
San [PRON, nsubj, head: envisage]
Francisco [PROPN, flat:name, head: San]
envisage [VERB, ROOT, head: envisage]
d' [ADP, case, head: interdire]
interdire [NOUN, obl:arg, head: envisage]
les [DET, det, head: robots]
robots [NOUN, obj, head: envisage]
coursiers [ADJ, amod, head: robots]
sur [ADP, case, head: trottoirs]
les [DET, det, head: trottoirs]
trottoirs [NOUN, nmod, head: robots]


**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`. 

In [5]:
# Veuillez écrire votre code ici.
for token in doc:
    if token.dep_ in ["nsubj", "obj"]:
        print(f"{token.dep_} : {token.text} [{token.pos_}, {token.dep_}, head: {token.head.text}]")
        for t in token.subtree:
            print(f"\t{t.text} [{t.pos_}, {t.dep_}, head: {t.head.text}]")

nsubj : San [PRON, nsubj, head: envisage]
	San [PRON, nsubj, head: envisage]
	Francisco [PROPN, flat:name, head: San]
obj : robots [NOUN, obj, head: envisage]
	les [DET, det, head: robots]
	robots [NOUN, obj, head: envisage]
	coursiers [ADJ, amod, head: robots]
	sur [ADP, case, head: trottoirs]
	les [DET, det, head: trottoirs]
	trottoirs [NOUN, nmod, head: robots]


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

In [6]:
# Veuillez écrire votre réponse dans cette cellule.
# La colonne 7 donne l’identifiant du mot auquel le mot courant est rattaché (le "gouverneur"), ou 0 si c’est le mot racine.
# La colonne 8 indique le type de relation de dépendance (comme sujet, objet, etc.).
# La colonne 9 peut contenir des dépendances supplémentaires sous forme de paires (gouverneur, relation).

In [7]:
from spacy.tokens import DocBin, Doc
test_data = DocBin().from_disk("spacy_data/fr-ud-test.spacy")
# for doc in test_data.get_docs(nlp.vocab):  # exemple
#     for sent in doc.sents:
#         print(sent)

**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [8]:
# Veuillez écrire votre code ici.
sents = list(test_data.get_docs(nlp.vocab))
s2 = list(sents[1].sents)[6]

print(f"Phrase : {s2.text}")

Phrase : Trois ans plus tard, il tient un discours sur la crise.


**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [11]:
from spacy import displacy


In [12]:
# Veuillez écrire votre code ici.
from IPython.display import HTML, display

display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(s2, style="dep", jupyter=False)
)))

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [13]:
# Veuillez écrire votre code ici.
doc2 = nlp(s2.as_doc())
display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(doc2, style="dep", jupyter=False)
)))

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

In [14]:
# Veuillez écrire votre réponse ici.
with open("original_tree.svg", "w", encoding="utf-8") as f:
    f.write(displacy.render(s2, style="dep", jupyter=False))
    
with open("predicted_tree.svg", "w", encoding="utf-8") as f:
    f.write(displacy.render(doc2, style="dep", jupyter=False))

# On peut voir que la différence se trouve dans le groupe "ans plus tard il tient"
# L'arbre original analyse correctement "il" comme sujet de "tient", qui est la racine de la phrase.
# Dans l'arbre prédit, "il" est incorrectement rattaché à "tard", et "tient" n'est plus la racine.
# Cela fausse toute la structure sujet-verbe et la relation temporelle "ans plus tard".

original_tokens = [(token.text, token.dep_, token.head.text) for token in s2]
predicted_tokens = [(token.text, token.dep_, token.head.text) for token in doc2]

# Calculer le taux de correction
correct = sum(1 for g, p in zip(original_tokens, predicted_tokens) if g == p)
total = len(original_tokens)
accuracy = correct / total

print(f"Taux de correction : {accuracy:.2%}")

Taux de correction : 84.62%


**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [15]:
from spacy.scorer import Scorer
from spacy.training import Example

In [31]:
# Veuillez écrire votre code et votre réponse ici.
scorer = Scorer()

scores = scorer.score([Example(nlp(s2.as_doc()), s2.as_doc())])

print(f"UAS : {scores['dep_uas']:.3f}")
print(f"LAS : {scores['dep_las']:.3f}")

# Comparer avec le calcul manuel précédent
print(f"\nComparaison avec le calcul manuel précédent:")
# 0.846
print(f"Taux de correction calculé manuellement: {accuracy:.3f}")

# On trouve en effet un score différent.
# Le score obtenu avec `scorer.score()` est plus rigoureux car il s'appuie sur l'identifiant du mot gouverneur (head) (et pas juste son texte).

UAS : 0.818
LAS : 0.818

Comparaison avec le calcul manuel précédent:
Taux de correction calculé manuellement: 0.846


## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [ ]:
# Veuillez écrire votre code ici.
examples = []
# Parcourir tous les documents et phrases dans les données de test
for doc in test_data.get_docs(nlp.vocab):
    for sent in doc.sents:
        
        parsed_sent = nlp(sent.text)
        example = Example(sent.as_doc(), parsed_sent)
        examples.append(example)

scores = scorer.score(examples)


print(f"UAS : {scores['dep_uas']:.3f}")
print(f"LAS : {scores['dep_las']:.3f}")

print("Documentation fr_core_news_sm: UAS = 88%, LAS = 84%")

# On voit qu'ils sont bien moins bon...

# Le score UAS (0.760) reste assez proche de celui de la documentation (88 %) (tout est relatif ...), 
# ce qui montre que les dépendances sont globalement bien trouvées, même s’il y a un peu plus d’erreurs. 
# Cela peut venir du fait que le modèle a été entraîné sur un autre type de texte. Le découpage du texte en plusieurs parties a peut-être aussi fait perdre du contexte.

# Le score LAS (0.619), lui, est bien plus bas que celui de la documentation (84 %). 
# Cela montre que le modèle a plus de mal à bien choisir le type de relation entre les mots, surtout si le contexte manque ou si les phrases sont difficiles à analyser.

UAS : 0.760
LAS : 0.619

Comparaison avec les scores de la documentation:
Documentation fr_core_news_sm: UAS = 88%, LAS = 84%


**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [18]:
import pandas as pd

scores_per_type = scores["dep_las_per_type"]
df = pd.DataFrame.from_dict(scores_per_type, orient="index")
df.reset_index(inplace=True)
df.columns = ["Relation", "Precision", "Recall", "F1-Score"]
df = df.sort_values(by="F1-Score", ascending=False)
df = df.round(3)
print(df.to_string(index=False))
df_before = df.copy()


    Relation  Precision  Recall  F1-Score
         det      0.770   0.928     0.842
        case      0.775   0.854     0.813
          cc      0.794   0.797     0.795
  nsubj:pass      0.860   0.683     0.761
        mark      0.779   0.723     0.750
    aux:pass      0.925   0.628     0.748
         cop      0.741   0.720     0.730
       nsubj      0.676   0.757     0.714
      nummod      0.664   0.750     0.705
        root      0.795   0.617     0.695
      advmod      0.700   0.678     0.689
         obj      0.694   0.684     0.689
        amod      0.743   0.588     0.656
   flat:name      0.632   0.610     0.621
        nmod      0.594   0.539     0.565
       xcomp      0.613   0.489     0.544
   acl:relcl      0.547   0.500     0.522
       fixed      0.337   0.523     0.410
         acl      0.416   0.403     0.409
        iobj      0.560   0.318     0.406
       advcl      0.382   0.402     0.392
       ccomp      0.317   0.465     0.377
        conj      0.370   0.381   

## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [20]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [24]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.
!python -m spacy train config.cfg \
  --output ./myDEPparser1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --verbose

[2025-04-09 19:00:07,525] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: myDEPparser1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-04-09 19:00:12,583] [INFO] Set up nlp object from config
[2025-04-09 19:00:12,603] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-04-09 19:00:12,605] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-04-09 19:00:12,605] [INFO] Pipeline: ['tok2vec', 'parser', 'morphologizer']
[2025-04-09 19:00:12,610] [INFO] Created vocabulary
[2025-04-09 19:00:12,611] [INFO] Finished initializing nlp object
[2025-04-09 19:01:03,477] [INFO] Initialized pipeline components: ['tok2vec', 'parser', 'morphologizer']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-04-09 19:01:03,497] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-04-09 19:01:03,500] [DEBUG] Loading c

In [25]:
# Veuillez indiquer ici le nombre d'époques final. 
# L'entraînement du modèle s’est déroulé sur 7 époques complètes. On observe que les scores sur le jeu de validation (notamment DEP_UAS et DEP_LAS) ont progressivement augmenté jusqu’à environ la 5ᵉ ou 6ᵉ époque, puis se sont stabilisés.
# Cela indique que le modèle a atteint un plateau de performance, et que prolonger l’entraînement au-delà de 7 époques n’aurait probablement pas apporté d’amélioration significative.

# En analysant la progression du F1-score global (SCORE), on constate qu’il augmente régulièrement durant les premières époques, passant de 0.24 à 0.81.
# En se basant uniquement sur cette métrique, on peut conclure que l’amélioration significative s’arrête vers l’époque 2. 


**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [26]:
# Veuillez écrire votre code ici.

nlp2 = spacy.load("./myDEPparser1/model-best")
data_test = DocBin().from_disk("spacy_data/fr-ud-test.spacy")

scorer = Scorer().score([
    Example(
        nlp2(Doc(nlp2.vocab, [token.text for token in doc])),
        doc
    ) for doc in test_data.get_docs(nlp2.vocab)
])

print(f"UAS: {scorer['dep_uas']:.3f}")
print(f"LAS: {scorer['dep_las']:.3f}")

UAS: 0.880
LAS: 0.838


**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [27]:
# Veuillez écrire votre code ici.
scores_per_type = scorer["dep_las_per_type"]
df = pd.DataFrame.from_dict(scores_per_type, orient="index")
df.reset_index(inplace=True)
df.columns = ["Relation", "Precision", "Recall", "F1-Score"]
df = df.sort_values(by="F1-Score", ascending=False)
df = df.round(3)
print(df.to_string(index=False))
df_after = df.copy()


  Relation  Precision  Recall  F1-Score
       det      0.972   0.985     0.978
      case      0.921   0.967     0.943
  aux:caus      1.000   0.846     0.917
       aux      0.915   0.909     0.912
    nummod      0.889   0.877     0.883
      amod      0.869   0.894     0.881
       obj      0.876   0.883     0.879
        cc      0.876   0.880     0.878
      root      0.875   0.861     0.868
     nsubj      0.848   0.881     0.864
      mark      0.865   0.861     0.863
       cop      0.862   0.856     0.859
 flat:name      0.829   0.857     0.843
     xcomp      0.817   0.802     0.810
  aux:pass      0.754   0.868     0.807
      nmod      0.805   0.783     0.794
    advmod      0.778   0.782     0.780
nsubj:caus      1.000   0.600     0.750
 obj:agent      1.000   0.571     0.727
       obl      0.699   0.740     0.719
nsubj:pass      0.756   0.680     0.716
 acl:relcl      0.694   0.667     0.680
     ccomp      0.632   0.683     0.656
      conj      0.582   0.634     0.607


In [28]:
# Compter le nombre de F1-scores à 0 pour chaque DataFrame
def count_zero_f1(df):
    return (df["F1-Score"] == 0.0).sum()

f1_zeros_before = count_zero_f1(df_before)  # df_before = tableau AVANT entraînement
f1_zeros_after = count_zero_f1(df_after)    # df_after = tableau APRÈS entraînement

print(f"Nombre de F1-score à 0 avant entraînement : {f1_zeros_before}")
print(f"Nombre de F1-score à 0 après entraînement : {f1_zeros_after}")


Nombre de F1-score à 0 avant entraînement : 20
Nombre de F1-score à 0 après entraînement : 5


**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

In [ ]:
## Dans les résultats obtenus, on remarque qu’un certain nombre de relations de dépendances conservent un score F1 nul, même après l'entraînement du parser. 
# Afin de quantifier cette observation, on compte le nombre de labels avec un F1-score égal à 0 avant et après entraînement. 
# Cette comparaison nous permet de voir qu'avant il y avait 20 labels avec un F1-score nul, et après il n'y en a plus que 5.

# Concernant les 3 relations les moins bien reconnues, avant entraînement, il s’agissait de discourse, flat:foreign, et vocative (toutes avec un F1 = 0.000), ce qui indique une absence totale de reconnaissance.
# Avant l'entraînement du parser, les 3 relations de dépendance les mieux reconnues étaient det (F1 = 0.842), case (F1 = 0.813) et cc (F1 = 0.795). Ces labels sont fréquents et relativement simples à identifier, ce qui explique leurs bons scores initiaux. 
# Après entraînement, on observe une amélioration générale avec en tête aux:caus (F1 = 0.917), det (F1 = 0.978) et case (F1 = 0.943). Cela montre que le modèle a su mieux reconnaître certaines relations, notamment aux:caus qui passe à la 3e place.
# Après entraînement, ces mêmes labels restent en bas du classement, avec toujours un F1-score nul, montrant que le modèle n’a pas appris à les traiter — probablement en raison d’un manque de représentativité dans les données d’apprentissage.

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.